# 智能重新排序 -- 使用Reranker Model对每个检索块的相关性进行评分

智能重新排序的目的是帮助我们从海量文档中快速找到最相关的信息。它就像是给文档们排个队，让最有用的信息排在最前面。

### 工作原理
在RAG系统中，我们通常会先找到一些可能相关的文档，但这些文档的相关性可能不够精确。智能重新排序就是对这些文档进行再次评估，找出真正和我们的需求紧密相关的文档。

### 重排系统通常包括以下步骤：
- 初始检索器：使用向量搜索技术，找到一些可能相关的文档。
- 评分机制：使用Reranker Model给这些文档进行打分，看看它们和我们的需求有多匹配。
- 排序和选择逻辑：根据分数重新排序这些文档，挑选出最相关的几个。

首先，我们需要导入一些Python库，并从环境变量文件中加载必要的配置。

In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

from typing import List, Dict, Any
from FlagEmbedding import FlagReranker
from qdrant_client import QdrantClient

from langchain_qdrant import QdrantVectorStore
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings

#### 第一步：加载文档并提取文档内容
使用Docx2txtLoader类加载文档，这个类需要docx2txt库支持。

In [30]:
path = "../data/二十届三中全会.docx"
loader = Docx2txtLoader(path)
documents = loader.load()

#### 第二步：文本分块
将文本切割成小块

In [31]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
docs = text_splitter.split_documents(documents)
for doc in docs:
    doc.page_content = doc.page_content.replace('\t', ' ') 

#### 第三步：创建向量数据库
这次换成FlagEmbedding和HuggingFaceBgeEmbeddings来创建向量表示，并存储在本地的Qdrant数据库中。

In [42]:
model_name = "BAAI/bge-small-zh"
model_kwargs = {'device': 'cuda'}
# 计算余弦相似度
encode_kwargs = {'normalize_embeddings': True} 
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    prefer_grpc=True,
    collection_name="005"
)

08/31/2024 20:43:29 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: BAAI/bge-small-zh


如果已经创建了向量数据库，就直接通过下面的代码直接连接Qdrant向量数据库

In [46]:
client = QdrantClient(url="http://localhost:6333/")
embeddings = MiniMaxEmbeddings()
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="005",
    embedding=embeddings
)

#### 第四步：创建重排系统
创建一个重排系统，使用BAAI/bge-reranker-large 模型来评估文档的相关性进行评分，并返回最相关的文档。

In [54]:
def rerank_documents(query: str, initial_top_n: int = 15, top_n: int = 3) -> List[Dict[str, Any]]:
    # 先使用向量相似搜索找到一些可能相关的文档
    initial_docs = vectorstore.similarity_search(query, k=initial_top_n)
    # 将这些文档和查询语句组成一个列表，每个元素是一个包含查询和文档内容的列表
    sentence_pairs = [[query, passage.page_content] for passage in initial_docs]

    # 使用FlagReranker模型对这些文档进行重新排序;将use_fp16设置为True可以提高计算速度，但性能略有下降
    reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)
    # 计算每个文档的得分
    scores = reranker.compute_score(sentence_pairs)

    # 将得分和文档内容组成一个字典列表
    score_document = [{"score": score, "content": content} for score, content in zip(scores, initial_docs)]
    # 根据得分对文档进行排序，并返回前top_n个文档
    result = sorted(score_document, key=lambda x: x['score'], reverse=True)[:top_n]
    
    return result


#### 执行输出结果
最后，我们用一个实际的问题来测试我们的系统，看看它是否能够找到最相关的文档。

In [55]:
query = "什么是健全协商民主机制?"
reranked_docs = rerank_documents(query)
reranked_docs

d:\dev\pythonFile\RAG_course\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'score': 4.18359375,
  'content': Document(metadata={'source': '../data/二十届三中全会.docx', '_id': '5f5614d1-ed7c-4630-bb98-7439f0792744', '_collection_name': '005'}, page_content='（29）加强人民当家作主制度建设。坚持好、完善好、运行好人民代表大会制度。健全人大对行政机关、监察机关、审判机关、检察机关监督制度，完善监督法及其实施机制，强化人大预算决算审查监督和国有资产管理、政府债务管理监督。健全人大议事规则和论证、评估、评议、听证制度。丰富人大代表联系人民群众的内容和形式。健全吸纳民意、汇集民智工作机制。发挥工会、共青团、妇联等群团组织联系服务群众的桥梁纽带作用。\n\n（30）健全协商民主机制。发挥人民政协作为专门协商机构作用，健全深度协商互动、意见充分表达、广泛凝聚共识的机制，加强人民政协反映社情民意、联系群众、服务人民机制建设。完善人民政协民主监督机制。\n\n完善协商民主体系，丰富协商方式，健全政党协商、人大协商、政府协商、政协协商、人民团体协商、基层协商以及社会组织协商制度化平台，加强各种协商渠道协同配合。健全协商于决策之前和决策实施之中的落实机制，完善协商成果采纳、落实、反馈机制。')},
 {'score': -0.61572265625,
  'content': Document(metadata={'source': '../data/二十届三中全会.docx', '_id': '0a0efd15-20a3-49a0-a113-f88b5e733316', '_collection_name': '005'}, page_content='（31）健全基层民主制度。健全基层党组织领导的基层群众自治机制，完善基层民主制度体系和工作体系，拓宽基层各类组织和群众有序参与基层治理渠道。完善办事公开制度。健全以职工代表大会为基本形式的企事业单位民主管理制度，完善企业职工参与管理的有效形式。\n\n（32）完善大统战工作格局。完善发挥统一战线凝聚人心、汇聚力量政治作用的政策举措。坚持好、发展好、完善好中国新型政党制度。更好发挥党外人士作用，健全党外代表人士队伍建设制度。制定民族团结进步